# 4.3 结构化输出实战

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/forhow134/ai-coding-guide/blob/main/demos/04-prompt-engineering/structured_output.ipynb)

**预计 API 费用：~$0.01**

本 Notebook 演示如何让 LLM 输出可靠的 JSON，包括 JSON Mode 和 Structured Outputs。

In [ ]:
!pip install -q openai pydantic

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

## Experiment 1: Prompt 提示（Basics方法）
<!-- 实验 1：Prompt 提示（基础方法） -->

最简单的方法：在 Prompt 中要求返回 JSON。

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

invoice_text = """
发票抬头：北京科技有限公司
金额：¥12,580.00
日期：2026-02-15
发票号：20260215001
"""

prompt = f"""
从发票中提取信息，以 JSON 格式返回。

要求字段：
- company: 公司名称
- amount: 金额（数字，不带符号）
- date: 日期（YYYY-MM-DD 格式）
- invoice_number: 发票号

发票内容：
{invoice_text}

请直接返回 JSON，不要其他文字。
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
)

result = response.choices[0].message.content
print("AI 原始返回：")
print(result)
print()

# 尝试解析
try:
    # 可能需要去除 Markdown 代码块标记
    if result.startswith("```"):
        result = result.split("\n", 1)[1].rsplit("\n", 1)[0]
    
    data = json.loads(result)
    print("解析成功！")
    print(json.dumps(data, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print(f"解析失败：{e}")

**问题：**
- 可能返回 Markdown 代码块（```json...```）
- 字段名可能不一致
- 可能包含多余的说明文字

需要手动处理这些情况，代码复杂且不可靠。

## Experiment 2: JSON Mode（推荐方法）
<!-- 实验 2：JSON Mode（推荐方法） -->

OpenAI 在模型层面强制输出有效 JSON。

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

invoice_text = """
发票抬头：北京科技有限公司
金额：¥12,580.00
日期：2026-02-15
发票号：20260215001
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "你是一个数据提取助手，输出格式为 JSON。"
        },
        {
            "role": "user",
            "content": f"""
从发票中提取信息，返回 JSON 对象，字段：company, amount, date, invoice_number。

发票内容：
{invoice_text}
"""
        }
    ],
    response_format={"type": "json_object"},  # 关键：启用 JSON Mode
)

result = response.choices[0].message.content
print("AI 返回（原始字符串）：")
print(result)
print()

# Parse JSON（保证能成功）
data = json.loads(result)
print("解析后的数据：")
print(json.dumps(data, indent=2, ensure_ascii=False))
print()

# 使用提取的数据
print("提取的字段：")
print(f"公司：{data.get('company', 'N/A')}")
print(f"金额：{data.get('amount', 'N/A')}")
print(f"日期：{data.get('date', 'N/A')}")
print(f"发票号：{data.get('invoice_number', 'N/A')}")

**JSON Mode 的优势：**
- ✅ 保证返回有效的 JSON 格式
- ✅ 不会返回 Markdown 代码块
- ✅ 简化 Prompt 编写

**注意：**
- ⚠️ 不保证字段名完全一致（可能返回 `companyName` 而非 `company`）
- ⚠️ 需要在 Prompt 中明确要求 JSON 输出

## Experiment 3: Structured Outputs（最强方法）
<!-- 实验 3：Structured Outputs（最强方法） -->

用 JSON Schema 严格定义输出结构，100% 匹配。

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

invoice_text = """
发票抬头：北京科技有限公司
金额：¥12,580.00
日期：2026-02-15
发票号：20260215001
"""

# 定义 JSON Schema
invoice_schema = {
    "type": "object",
    "properties": {
        "company": {"type": "string", "description": "公司名称"},
        "amount": {"type": "number", "description": "金额（数字）"},
        "date": {"type": "string", "description": "日期，格式 YYYY-MM-DD"},
        "invoice_number": {"type": "string", "description": "发票号"}
    },
    "required": ["company", "amount", "date", "invoice_number"],
    "additionalProperties": False
}

response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",  # 必须是支持 Structured Outputs 的模型
    messages=[
        {
            "role": "system",
            "content": "你是一个数据提取助手。"
        },
        {
            "role": "user",
            "content": f"从发票中提取信息：\n\n{invoice_text}"
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "invoice_extraction",
            "schema": invoice_schema,
            "strict": True  # 严格模式
        }
    }
)

result = json.loads(response.choices[0].message.content)
print("提取结果（严格匹配 Schema）：")
print(json.dumps(result, indent=2, ensure_ascii=False))
print()

# 字段名 100% 匹配，类型 100% 正确
print("验证：")
print(f"company 类型：{type(result['company']).__name__}")
print(f"amount 类型：{type(result['amount']).__name__}")
print(f"date 类型：{type(result['date']).__name__}")
print(f"invoice_number 类型：{type(result['invoice_number']).__name__}")

**Structured Outputs 的优势：**
- ✅ 严格匹配 JSON Schema（字段名、类型、必填项）
- ✅ 不会有多余字段
- ✅ 支持复杂嵌套结构
- ✅ 生产环境的首选方案

## Experiment 4: Pydantic 集成（最佳开发体验）
<!-- 实验 4：Pydantic 集成（最佳开发体验） -->

用 Pydantic 定义数据模型，自动生成 Schema，类型安全。

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
import json

client = OpenAI()

# 用 Pydantic 定义数据模型
class Invoice(BaseModel):
    company: str = Field(description="公司名称")
    amount: float = Field(description="金额（数字）")
    date: str = Field(description="日期，格式 YYYY-MM-DD")
    invoice_number: str = Field(description="发票号")

invoice_text = """
发票抬头：北京科技有限公司
金额：¥12,580.00
日期：2026-02-15
发票号：20260215001
"""

# Pydantic 自动生成 JSON Schema
response = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "system",
            "content": "你是一个数据提取助手。"
        },
        {
            "role": "user",
            "content": f"从发票中提取信息：\n\n{invoice_text}"
        }
    ],
    response_format=Invoice,  # 直接传 Pydantic 模型
)

# Return的是 Pydantic 对象，可以直接使用
invoice = response.choices[0].message.parsed

print("提取的 Pydantic 对象：")
print(f"公司：{invoice.company}")
print(f"金额：{invoice.amount}")
print(f"日期：{invoice.date}")
print(f"发票号：{invoice.invoice_number}")
print()

# 转换为字典或 JSON
print("转换为 JSON：")
print(invoice.model_dump_json(indent=2))

**Pydantic 的优势：**
- ✅ 不用手写 JSON Schema
- ✅ IDE 自动补全（类型提示）
- ✅ 自动数据验证
- ✅ 代码更简洁、更安全

## Experiment 5: 复杂结构 - 嵌套对象和数组
<!-- 实验 5：复杂结构 - 嵌套对象和数组 -->

Pydantic 支持复杂的数据结构。

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List

client = OpenAI()

# 定义嵌套模型
class OrderItem(BaseModel):
    name: str = Field(description="商品名称")
    quantity: int = Field(description="数量")
    price: float = Field(description="单价")

class Order(BaseModel):
    order_id: str = Field(description="订单号")
    customer: str = Field(description="客户名称")
    items: List[OrderItem] = Field(description="订单明细")
    total: float = Field(description="总金额")

order_text = """
订单号：ORD-20260215-001
客户：张三
商品明细：
1. Python 入门书籍 × 2 @ ¥89.00
2. JavaScript 高级编程 × 1 @ ¥128.00
总计：¥306.00
"""

response = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "user",
            "content": f"从订单中提取信息：\n\n{order_text}"
        }
    ],
    response_format=Order,
)

order = response.choices[0].message.parsed

print("订单信息：")
print(f"订单号：{order.order_id}")
print(f"客户：{order.customer}")
print(f"\n商品明细：")
for i, item in enumerate(order.items, 1):
    print(f"  {i}. {item.name} × {item.quantity} @ ¥{item.price}")
print(f"\n总计：¥{order.total}")
print()

# 计算验证
calculated_total = sum(item.quantity * item.price for item in order.items)
print(f"计算验证：{calculated_total} == {order.total} ? {calculated_total == order.total}")

## Experiment 6: Practice案例 - 用户评论Analysis
<!-- 实验 6：实战案例 - 用户评论分析 -->

从用户评论中提取结构化信息。

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Literal, Dict

client = OpenAI()

class Review(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="情感倾向")
    rating: int = Field(description="评分（1-5 星）", ge=1, le=5)
    aspects: Dict[str, str] = Field(description="各方面评价，如 {'质量': '好', '价格': '贵'}")
    summary: str = Field(description="一句话总结")

review_text = """
买了这款手机用了一周，整体还不错。
屏幕很清晰，拍照效果也不错，就是价格有点贵。
电池续航一般，玩游戏的话一天要充两次电。
总体来说物有所值吧，给 4 星。
"""

response = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "system",
            "content": "你是一个评论分析助手，擅长从用户评论中提取结构化信息。"
        },
        {
            "role": "user",
            "content": f"分析这条评论：\n\n{review_text}"
        }
    ],
    response_format=Review,
)

review = response.choices[0].message.parsed

print("评论分析结果：")
print(f"情感：{review.sentiment}")
print(f"评分：{review.rating} 星")
print(f"\n各方面评价：")
for aspect, opinion in review.aspects.items():
    print(f"  - {aspect}：{opinion}")
print(f"\n总结：{review.summary}")

## Experiment 7: 批量处理
<!-- 实验 7：批量处理 -->

用结构化输出处理多条数据。

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Literal

client = OpenAI()

class Sentiment(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="情感倾向")
    confidence: float = Field(description="置信度（0-1）", ge=0, le=1)

# 批量评论
comments = [
    "这个产品太棒了！",
    "质量太差，非常失望。",
    "还行吧，凑合用。",
    "价格有点贵，但质量不错。",
]

print("批量情感分析：")
print("="*80)

for comment in comments:
    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "user",
                "content": f"分析情感：{comment}"
            }
        ],
        response_format=Sentiment,
    )
    
    result = response.choices[0].message.parsed
    print(f"评论: {comment}")
    print(f"情感: {result.sentiment} (置信度: {result.confidence:.2f})")
    print()

## 方案选择指南

```
需要结构化输出？
  ↓ 是
是否需要严格的字段名和类型？
  ↓ 是 → Structured Outputs + Pydantic (生产环境)
  ↓ 否 → JSON Mode (快速开发)
```

| 场景 | 推荐方案 |
|-----|----------|
| **生产环境数据提取** | Structured Outputs + Pydantic |
| **简单 JSON 输出** | JSON Mode |
| **快速原型** | Prompt 提示 |
| **复杂嵌套结构** | Pydantic |
| **需要数据验证** | Pydantic |

## Hands-on Exercises
<!-- 动手练习 -->

1. **创建简历解析器**：用 Pydantic 定义简历模型（姓名、教育、工作经历），从文本中提取
2. **产品评论数据库**：批量处理 10 条评论，存入 JSON 文件
3. **对比成本**：测量 Prompt 提示 vs JSON Mode vs Structured Outputs 的 token 消耗差异
4. **错误处理**：测试当输入数据不完整时，Structured Outputs 如何处理

## Key Takeaways
<!-- 关键要点总结 -->

1. **Prompt 提示**：最简单，但不可靠，需要手动处理格式问题
2. **JSON Mode**：保证有效 JSON，但不保证字段名，适合快速开发
3. **Structured Outputs**：严格匹配 JSON Schema，生产环境首选
4. **Pydantic 集成**：自动生成 Schema，类型安全，开发体验最好
5. **选择原则**：生产环境用 Structured Outputs + Pydantic，原型用 JSON Mode

---

## 🎉 恭喜完成入门篇！

你已经掌握：
- ✅ Prompt 基础和进阶技巧
- ✅ Few-shot、Chain-of-Thought、Self-Consistency
- ✅ 结构化输出（JSON Mode、Structured Outputs、Pydantic）

**下一步：** 进入 [第 5 章：AI 编程工具全景](/05-ai-coding-tools/)，学习如何用 Cursor、GitHub Copilot 等工具提升开发效率。